In [7]:
import whisperx
import gc 

device = "cuda" 
batch_size = 8 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

# 1. Transcribe with original whisper (batched)
model = whisperx.load_model("large-v3", device, compute_type=compute_type)



Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../../data/storage025/yzhongenv/lib/python3.10/site-packages/whisperx/assets/pytorch_model.bin`


No language specified, language will be first be detected for each audio file (increases inference time).
Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.4.0+cu121. Bad things might happen unless you revert torch to 1.x.


In [12]:

# save model to local path (optional)
# model_dir = "/path/"
# model = whisperx.load_model("large-v2", device, compute_type=compute_type, download_root=model_dir)

audio_path = '/data/storage1t/PC-GITA/PC-GITA//sentences2/2_juan/non-normalized/pd/AVPEPUDEA0016_juan.wav'
audio = whisperx.load_audio(audio_path)
result = model.transcribe(audio, batch_size=batch_size, language='es')
print(result["segments"]) # before alignment

# delete model if low on GPU resources
# import gc; gc.collect(); torch.cuda.empty_cache(); del model


[{'text': ' Cuando se rompió una pierna cuando iba en la moto.', 'start': 0.031, 'end': 1.837}]


In [15]:


# 2. Align whisper output
model_a, metadata = whisperx.load_align_model('es', device=device)
print(metadata) # metadata contains the word timestamps
gt_text = [{'text': 'Juan se rompió una pierna cuando iba en la moto.', 'start': 0.031, 'end': 1.837}]


result = whisperx.align(gt_text, model_a, metadata, audio, device, return_char_alignments=True)

# print(result["segments"]) # after alignment

print(result)

{'language': 'es', 'dictionary': {'-': 0, '|': 1, 'e': 2, 'a': 3, 'o': 4, 's': 5, 'n': 6, 'r': 7, 'i': 8, 'l': 9, 'd': 10, 'c': 11, 't': 12, 'u': 13, 'p': 14, 'm': 15, 'b': 16, 'q': 17, 'y': 18, 'g': 19, 'v': 20, 'h': 21, 'ó': 22, 'f': 23, 'í': 24, 'á': 25, 'j': 26, 'z': 27, 'ñ': 28, 'é': 29, 'x': 30, 'ú': 31, 'k': 32, 'w': 33, 'ü': 34}, 'type': 'torchaudio'}
[{'start': 0.031, 'end': 1.673, 'text': 'Juan se rompió una pierna cuando iba en la moto.', 'words': [{'word': 'Juan', 'start': 0.031, 'end': 0.216, 'score': 0.181}, {'word': 'se', 'start': 0.257, 'end': 0.298, 'score': 0.19}, {'word': 'rompió', 'start': 0.318, 'end': 0.524, 'score': 0.313}, {'word': 'una', 'start': 0.544, 'end': 0.647, 'score': 0.15}, {'word': 'pierna', 'start': 0.667, 'end': 0.934, 'score': 0.306}, {'word': 'cuando', 'start': 0.975, 'end': 1.139, 'score': 0.22}, {'word': 'iba', 'start': 1.16, 'end': 1.221, 'score': 0.043}, {'word': 'en', 'start': 1.242, 'end': 1.303, 'score': 0.322}, {'word': 'la', 'start': 1.32